In [ ]:
from utils._model import build_and_compile_model, train_model_with_kfold, train_models_with_kfold, display_summaries, build_and_compile_model_custom
from utils._evaluation import evaluate_models, generate_reports, generate_single_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from utils._image import display_images_from_subfolders
from sklearn.model_selection import train_test_split
from utils._data import get_subfolders, load_data
from tensorflow.keras.utils import to_categorical
from sklearn.utils import shuffle
from utils._plot import plot

In [ ]:
import yaml

with open("config.yaml", 'r') as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
labels = ['glioma', 'meningioma', 'no_tumor', 'pituitary']
testing_dir = 'Testing/'
training_dir = 'Training/'

In [ ]:
subfolders = get_subfolders(training_dir)
display_images_from_subfolders(training_dir, subfolders)

## Creating training and testing data frames

In [ ]:
X_train, y_train = load_data(training_dir, labels)
X_test, y_test = load_data(testing_dir, labels)

In [ ]:
X_train, y_train = shuffle(X_train, y_train, random_state=42)

y_train = to_categorical(y_train) # One-hot encoding of integer labels
y_test = to_categorical(y_test)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42) # 20% of the data is validation data

In [ ]:
image_gen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)

image_gen.fit(X_train)

## Model Creation

In [ ]:
models = {
    'EfficientNet': build_and_compile_model(architecture='EfficientNet', config=config),
    'DenseNet': build_and_compile_model(architecture='DenseNet', config=config),
    'MobileNet': build_and_compile_model(architecture='MobileNet', config=config),
    'ResNet50': build_and_compile_model(architecture='ResNet50', config=config)
}

display_summaries(models)

## Model Training

Instead of training each model once on a static train/validation split, each model will now be trained and evaluated n_splits times. Each time it will be with a different train/validation set derived from the original X_train and y_train. In other words, we split the data into k subsets and validate the model k times.

In [ ]:
# all_metrics = train_models_with_kfold(models, X_train, y_train, image_gen=image_gen, config=config)

evaluate_models(models, X_test, y_test)
generate_reports(all_metrics, X_test, y_test, models)

## Create custom Convolutional Neural Network

In [ ]:
model = build_and_compile_model_custom(num_conv_blocks=5, num_dense_layers=2, config=config)
model.summary()

In [ ]:
metrics = train_model_with_kfold(model, 'CustomNet', X_train, y_train, image_gen, n_splits=5, config=config)
generate_single_report('CustomNet', metrics, model, X_test, y_test)